In [ ]:
import os
import tensorflow as tf
import numpy as np
import json
import pandas as pd
from PIL import Image
from datetime import datetime
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import preprocess_input

# method used for splitting the dataset into training and validation set
from sklearn.model_selection import train_test_split 

##Mounting the Drive

In [ ]:
drive.mount('/content/drive/')

cwd = os.getcwd()

dataset_dir = os.path.join(cwd, '/content/drive/My Drive/no_mask_detector/dataset')

decide_class_indices = True
if decide_class_indices:
  classes = [ '0',         # no_person
              '1',         # all_the_people
              '2' ]        # someone
else:
  classes = None

Mounted at /content/drive/


##Data Augmentation

In [ ]:
# Data augmentation

apply_data_augmentation = True

# Create training ImageDataGenerator object with data augmentation
if apply_data_augmentation:
  train_data_gen = ImageDataGenerator(rotation_range=20,
                                      width_shift_range=20,
                                      height_shift_range=20,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='constant',
                                      cval=0,
                                      preprocessing_function=preprocess_input)
else:
  train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Create validation ImageDataGenerator object
valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

##Training and Validation sets

In [ ]:
# Folders for the train and test set

training_dir = os.path.join(dataset_dir, 'training')
test_dir = os.path.join(dataset_dir, 'test')

In [ ]:
# Dividing the dataset into training and validation set

bs = 16
num_classes = 3

# We decide to set the image width and height to 612x408 cause we saw it was the most common image size on both the train and test set
img_w = 612
img_h = 408

# Set the random seed
SEED = 1234
tf.random.set_seed(SEED)


# Loading json file into a pandas dataframe
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)

dataframe = pd.DataFrame(dic.items())
dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)
dataframe["class"] = dataframe["class"].astype(str)

# Create the training and validation dataframe
train_dataframe = pd.DataFrame()
valid_dataframe = pd.DataFrame()

# Splitting the dataset into training and validation with respectively 90% and 10% of the entire samples
train_dataframe, valid_dataframe = train_test_split(dataframe, test_size = 0.1, shuffle= True)


# Training
train_gen = train_data_gen.flow_from_dataframe(train_dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               target_size=(img_w,img_h),
                                               shuffle=True,
                                               seed=SEED)

# Validation
valid_gen = valid_data_gen.flow_from_dataframe(valid_dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               target_size=(img_w,img_h),
                                               shuffle=False,
                                               seed=SEED)

In [ ]:
# Creating the training set and validation set from the generators

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_w, img_h, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()


valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_w, img_h, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

##Model Creation

In [ ]:
# Loading the ResNet152V2 model

# We used the GAP to limit the overfitting by reducing the number of parameters of our model
# We didn't include the top layers to make us build our own fully connected layers
resnet = tf.keras.applications.ResNet152V2(include_top=False,
                                          weights="imagenet",
                                          input_shape=(img_w, img_h, 3),
                                          pooling = 'avg')

resnet.summary()

In [ ]:
# Create Model

# Fine tuning

finetuning = True

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in resnet.layers[:freeze_until]:
        layer.trainable = False
else:
    resnet.trainable = False
    
model = tf.keras.Sequential()
model.add(resnet)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(.4))

# Kernel_regularizer to have another method for limiting the overfitting
model.add(tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer='l2'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()

In [ ]:
# Optimization parameters


# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)


# Validation metrics
metrics = ['accuracy']


# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Callbacks
callbacks = []


#Early Stopping
early_stop = True
if early_stop:
    
    # We decide to put the restore_best_weights of the EarlyStopping callback to true in order to have the best performing model 
    # after the training phase and not the last one before stopping
    
    # We put the patience to 8 to wait a little bit more for hypothetical small decreasing on the loss of the validation set
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    callbacks.append(es_callback)

##Model Training

In [ ]:
# Model Training

# The 2 quantities below are the step size per epoch for respectively the training set and validation set
# We chose those quantities to make the steps depending on the number of samples on the set and on the batch size
STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size

model.fit(x=train_dataset,
          epochs=200,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=valid_dataset,
          validation_steps=STEP_SIZE_VALID,
          callbacks=callbacks)

##Model Prediction and save results

In [ ]:
# Model Prediction
image_filenames_test = next(os.walk(test_dir))[2]

# dictionary for the predictions made at each iteration of the following for cycle
predictions = {}
for image_filename in image_filenames_test:

  # Image loading and converting to RGB mode
  img = Image.open(os.path.join(test_dir,image_filename)).convert('RGB')

  # Resizing it to make it suitable to the model structure
  img = img.resize((img_h,img_w))
  img_array = np.array(img)
  img_array = np.expand_dims(img_array, 0)

  # Preprocessing to make it suitable to the resnetV2 architecture
  img_array = preprocess_input(img_array)

  prediction = model.predict(img_array)

  # Taking the argmax of the predictions made to have the class predicted
  predictions[image_filename] = np.argmax(np.matrix(prediction))


# Function to create the CSV file with the results
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


# Create the CSV file from the predictions made and save it on our folder on Drive
create_csv(predictions, '/content/drive/My Drive/no_mask_detector/results')